<a href="https://colab.research.google.com/github/DevaSunder/Coding-works/blob/main/CompSecu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SQL Injection Implementation and Taint Analysis to solve the problem

#### The **pysqlite3** package is a Python interface for SQLite databases. It allows you to interact with SQLite databases using Python code. 

In [ ]:
! pip install pysqlite3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.0-cp310-cp310-linux_x86_64.whl size=162340 sha256=819011e542fe12e1b5df189f3fbf6a92a6edc734c34508775091954d977ca4f5
  Stored in directory: /root/.cache/pip/wheels/63/4e/34/5a90d9770b5fbf518f7a5da144c558f1c545935ad2eec9cf82
Successfully built pysqlite3


### Drop any **existing** tables

In [7]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Drop the table if it exists
c.execute("DROP TABLE IF EXISTS users")

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


### Creating new table and inserting the **username and passwords** for all the users

In [8]:
import sqlite3

# Connect to the database (creates a new database if it doesn't exist)
conn = sqlite3.connect('example.db')

# Create a cursor object to execute SQL queries
c = conn.cursor()

# Create a table
c.execute('''CREATE TABLE users
             (id INTEGER PRIMARY KEY, username TEXT, password TEXT)''')

# Insert some data into the table
c.execute("INSERT INTO users (username, password) VALUES ('DevaSunder', 'Pass123word')")
c.execute("INSERT INTO users (username, password) VALUES ('Nithin', 'Speednithin')")
c.execute("INSERT INTO users (username, password) VALUES ('Arthi', 'Flower123base')")
c.execute("INSERT INTO users (username, password) VALUES ('Sarvesh', 'Ronaldo_Rocks')")
c.execute("INSERT INTO users (username, password) VALUES ('Shree', 'Sun999Flower')")
c.execute("INSERT INTO users (username, password) VALUES ('Maxet', 'Vikas123')")
c.execute("INSERT INTO users (username, password) VALUES ('English', 'Dave9489')")
c.execute("INSERT INTO users (username, password) VALUES ('dave', 'passwordabc')")
c.execute("INSERT INTO users (username, password) VALUES ('Karadi', 'Peter')")

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


### Displaying the current database

In [13]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Execute a SELECT query to retrieve all rows from the table
c.execute("SELECT * FROM users")

# Fetch all rows and print them
rows = c.fetchall()
for row in rows:
    print(row)

# Close the database connection
conn.close()


(1, 'DevaSunder', 'Pass123word')
(2, 'Nithin', 'Speednithin')
(3, 'Arthi', 'Flower123base')
(4, 'Sarvesh', 'Ronaldo_Rocks')
(5, 'Shree', 'Sun999Flower')
(6, 'Maxet', 'Vikas123')
(7, 'English', 'Dave9489')
(8, 'dave', 'passwordabc')
(9, 'Karadi', 'Peter')


### *Normal* SQL Query to check if the **username and password** entered matches with the database

In [24]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('example.db')
c = conn.cursor()

# Normal query
username = 'DevaSunder'
password = 'Pass123word'
query = "SELECT * FROM users WHERE username = ? AND password = ?"
params = (username, password)
c.execute(query, params)
print(c.fetchall())




[(1, 'DevaSunder', 'Pass123word')]


### Tainted SQL Query where the **username and password** are *tainted* . We can see below that the system logs the user in irrespective of the username and password enterned. We can see that the username is given as **"' OR 1=1 --"**

In [28]:
import sqlite3

# Establish the database connection
conn = sqlite3.connect("example.db")
c = conn.cursor()

# Get user input
username = input("Enter your username: ")
password = input("Enter your password: ")

username = "' OR 1=1 --" ## BAD Query
# Construct the SQL query (vulnerable to SQL injection)
query = "SELECT * FROM users WHERE username = '" + username + "' AND password = '" + password + "'"

# Execute the query
c.execute(query)

# Check if the user is authenticated
result = c.fetchall()
if len(result) > 0:
    print("Authentication successful!")
else:
    print("Authentication failed!")

# Close the database connection
conn.close()


Enter your username: dcsdew
Enter your password: hackea
Authentication successful!


### We have implemented tainted analysis where the query entering is checked for any bad modification.

#### NORMAL INPUT

In [29]:
import re

def is_tainted(input_str):
    # Define the patterns for SQL injection detection
    patterns = [
        r"(\bOR\b|\bAND\b|\bUNION\b|\bSELECT\b|\bINSERT\b|\bUPDATE\b|\bDELETE\b|\bDROP\b)",
        r"[\"'][^\"'\\]*(?:\\.[^\"'\\]*)*[\"']"
    ]

    for pattern in patterns:
        if re.search(pattern, input_str, re.IGNORECASE):
            return True
    
    return False

# Get user input
username = input("Enter your username: ")
password = input("Enter your password: ")

# Perform taint analysis
if is_tainted(username) or is_tainted(password):
    print("Potential SQL injection detected!")
else:
    print("No potential SQL injection detected.")


Enter your username: deasd
Enter your password: g
No potential SQL injection detected.


#### TAINTED INPUT

In [31]:
import re

def is_tainted(input_str):
    # Define the patterns for SQL injection detection
    patterns = [
        r"(\bOR\b|\bAND\b|\bUNION\b|\bSELECT\b|\bINSERT\b|\bUPDATE\b|\bDELETE\b|\bDROP\b)",
        r"[\"'][^\"'\\]*(?:\\.[^\"'\\]*)*[\"']"
    ]

    for pattern in patterns:
        if re.search(pattern, input_str, re.IGNORECASE):
            return True
    
    return False

# Get user input
username = input("Enter your username: ")
password = input("Enter your password: ")

# Perform taint analysis
if is_tainted(username) or is_tainted(password):
    print("Potential SQL injection detected!")
else:
    print("No potential SQL injection detected.")


Enter your username: 'admin' #'
Enter your password: asdac
Potential SQL injection detected!
